# Задача 1

Найти оптимум для задачи из классной работы, с использованием квадратичной функции потерь.  
Необходимо использовать алгоритм BFGS для поиска минимума.

Сравнить результаты (какие получились коэффициенты, какое получилось минимальное значение функции потерь).

In [215]:
import numpy as np
import pandas as pd
from scipy import optimize
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,
                                    LogisticRegression)
%matplotlib inline

In [164]:
data = np.array([
    [3, 30000, 2],
    [10, 65000, 0],
    [6, 15000, 1],
    [12, 53000, 0],
    [1, 70000, 0],
    [3, 45000, 1],
    [4, 25000, 0],
    [1, 20000, 2]
])

In [169]:
def func_mse(a):
    errors = []
    for row in data:
        errors.append((row[0]*a[0] + row[1]*a[1] + a[2] - row[2])**2)
#         print((row[0]*a[0] + row[1]*a[1] + a[2] - row[2])**2)
    return sum(errors)

In [170]:
res1 = optimize.minimize(func_mse, [0, 0, 0], method='BFGS')

print(res1)

      fun: 2.64728351521199
 hess_inv: array([[ 4.72573509e-03, -2.70814623e-07, -1.26973337e-02],
       [-2.70814623e-07,  1.78518202e-10, -5.85362454e-06],
       [-1.26973337e-02, -5.85362454e-06,  3.62343537e-01]])
      jac: array([-9.56654549e-06, -8.94069672e-08,  5.36441803e-07])
  message: 'Optimization terminated successfully.'
     nfev: 28
      nit: 5
     njev: 7
   status: 0
  success: True
        x: array([-7.81331800e-02, -2.23668142e-05,  2.04372605e+00])


In [168]:
func_mse([-7.81331800e-02, -2.23668142e-05,  2.04372605e+00])

0.7424888309221028
0.036652594393460934
0.057324214264509094
0.006290593624256548
0.15993270787684746
0.03888000327245661
1.3736378539278506
0.23207671693237528


2.6472835152138594

In [176]:
RMSE1 = np.sqrt(res1.fun)
print(RMSE1, res1.fun)

1.6270474840065332 2.64728351521199


In [1]:
abs(10*-7.81331800e-02 + 65000*-2.23668142e-05 + 2.04372605e+00)

0.191448673

# Задача 2

К заданию приложен файл `Folds5x2_pp.xlsx`, в котором содержится следующая информация:

Данные содержат около 10 тыс наблюдений об электростанции комбинированного цикла, когда электростанция работала с полной загрузкой. У нас есть следующие факторы (усредненные за час):

1. Температура (АТ)
2. Давление окружающей среды (АР)
3. Вытяжной вакуум (V)
4. Относительная влажность (RH)

Каждая из величин представлена в своих единицах измерения:

- Temperature (T) in the range 1.81°C and 37.11°C,
- Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
- Relative Humidity (RH) in the range 25.56% to 100.16%
- Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
- Net hourly electrical energy output (EP) 420.26-495.76 MW

Необходимо предсказать количество отпускаемой электроэнергии в час (EP).

Помимо прочего, необходимо проверить работу алгоритма на примере, которого нет в обучающей выборке.

In [220]:
df = pd.read_excel("Folds5x2_pp.xlsx")

In [221]:
df

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90
...,...,...,...,...,...
9563,16.65,49.69,1014.01,91.00,460.03
9564,13.19,39.18,1023.67,66.78,469.62
9565,31.32,74.33,1012.92,36.48,429.57
9566,24.48,69.45,1013.86,62.39,435.74


In [40]:
X = df.values[:, :-1]
y = df.values[:, -1]

In [41]:
from sklearn.model_selection import  train_test_split

In [42]:
X.shape,y.shape

((9568, 4), (9568,))

In [43]:
X_train,X_test,y_train,y_test = train_test_split(X, y,
                                                 test_size=0.33,
                                                 random_state=42,
                                                 shuffle = True)

In [44]:
X_train.shape, X_test.shape

((6410, 4), (3158, 4))

In [45]:
y_train

array([432.81, 494.67, 442.32, ..., 435.06, 471.13, 472.43])

In [49]:
lr = LinearRegression()

In [50]:
lr.fit(X_train, y_train)

LinearRegression()

In [51]:
lr.coef_

array([-2.00154533, -0.22856444,  0.04996956, -0.16072994])

In [69]:
X_test[0]

array([  19.64,   48.06, 1014.81,   74.96])

In [70]:
lr.predict([X_test[0]])

array([455.62963361])

In [71]:
y_test[0]

455.27

In [72]:
abs(lr.predict([X_test[0]])[0] - y_test[0])

0.3596336081246818

In [73]:
lr.predict([[14.96, 41.76, 1024.07, 73.17]])

array([467.18724646])

In [211]:
def func_mse_2(a):
    errors = []
    for andex, row in df.head(10).iterrows():
        errors.append((row['AT']*a[0] + row['V']*a[1] + row['AP']*a[2] + row['RH']*a[3] + a[4] - row['PE'])**2)
#         print((row['AT']*a[0] + row['V']*a[1] + row['AP']*a[2] + row['RH']*a[3] + a[4] - row['PE'])**2)
    return sum(errors)

In [212]:
res2 = optimize.minimize(func_mse_2, [0, 2, 3, 1, 2], method='BFGS')

print(res2)

      fun: 47.5540658686752
 hess_inv: array([[ 6.77787778e-03, -4.46567001e-03,  1.24959582e-04,
         2.52235290e-04, -3.74760992e-02],
       [-4.46567001e-03,  4.09432980e-03,  9.19382267e-04,
         3.45781234e-04, -1.08399854e+00],
       [ 1.24959582e-04,  9.19382267e-04,  4.31440457e-03,
         1.13047360e-03, -4.50851809e+00],
       [ 2.52235290e-04,  3.45781234e-04,  1.13047360e-03,
         5.83832846e-04, -1.21014809e+00],
       [-3.74760992e-02, -1.08399854e+00, -4.50851809e+00,
        -1.21014809e+00,  4.71788029e+03]])
      jac: array([ 4.29153442e-06,  8.58306885e-06, -4.76837158e-06,  4.76837158e-06,
        9.53674316e-07])
  message: 'Optimization terminated successfully.'
     nfev: 162
      nit: 15
     njev: 27
   status: 0
  success: True
        x: array([-2.24520013e+00, -3.43851695e-01, -4.02046392e-01, -3.36043132e-01,
        9.49353316e+02])


In [213]:
for i in range(5):
    for j in range(5):
        for x in range(5):
            for y in range(5):
                for z in range(5):
                    print(i, j, x, y, z)
                    res2 = optimize.minimize(func_mse_2, [i, j, x, y, z], method='BFGS')
                    if res2.success == True:
                        
                        print(res2)

0 0 0 0 0
0 0 0 0 1
0 0 0 0 2
0 0 0 0 3
0 0 0 0 4
0 0 0 1 0
0 0 0 1 1
0 0 0 1 2
0 0 0 1 3
0 0 0 1 4
0 0 0 2 0
0 0 0 2 1
0 0 0 2 2
0 0 0 2 3
0 0 0 2 4


KeyboardInterrupt: 

In [234]:
for andex, row in df.head(10).iterrows():
        print(row['AT']*-2.24520013e+00 + row['V']*-3.43851695e-01 + row['AP']*-4.02046392e-01 \
              + row['RH']*-3.36043132e-01 + 9.49353316e+02,
              row['PE'])

465.09395064811997 463.26
441.21344407515994 444.37
486.4342962435 488.56
450.89116944024 446.48
473.9400444188999 473.9
443.2206895923 443.67
465.59439775458 467.35
480.63834521034994 478.42
476.34551820904 475.98
476.11814196968 477.5


In [209]:
16.65*-2.24520013e+00 + 49.69*-3.43851695e-01 + 1014.01*-4.02046392e-01 + 91.00*-3.36043132e-01 + 9.49353316e+02


456.62575614703